In [1]:
import anndata
import torch
import stPlus
import os
import random
import warnings

import squidpy as sq
import numpy as np
import scanpy as sc
import pandas as pd

from sklearn.model_selection import KFold
from transpa.eval_util import calc_corr
from transpa.util import expTransImp, leiden_cluster, compute_autocorr
from benchmark import SpaGE_impute, Tangram_impute


warnings.filterwarnings('ignore')

seed = 10
device = torch.device("cuda:1") if torch.cuda.is_available() else torch.device("cpu")

In [2]:
spa_adata = sc.read_h5ad("../../data/ST/seqfish/seqfish_data.h5ad")
scrna_adata = sc.read_h5ad("../../data/scRNAseq/seqfish/scRNAseq_seqfish.h5ad")
classes, ct_list = leiden_cluster(scrna_adata)
cls_key = 'leiden'
sc.pp.normalize_total(spa_adata)
sc.pp.normalize_total(scrna_adata)
sc.pp.log1p(spa_adata)
sc.pp.log1p(scrna_adata)

var_name = scrna_adata.var_names.values.copy()
var_name[np.argmax((scrna_adata.var_names == "Prkcdbp"))] = "Cavin3"
scrna_adata.var_names = var_name

scrna_adata.obs[cls_key] = classes

In [3]:
raw_spatial_df  = pd.DataFrame(spa_adata.X.toarray(), columns=spa_adata.var_names)
raw_spatial_df.to_csv('../../output/seqfish_raw.csv')
raw_scrna_df    = pd.DataFrame(scrna_adata.X.toarray(), columns=scrna_adata.var_names).astype(pd.SparseDtype("float32", 0))
raw_shared_gene = np.intersect1d(raw_spatial_df.columns, raw_scrna_df.columns)
raw_spatial_df.shape, raw_scrna_df.shape, raw_shared_gene.shape,

((57536, 351), (32844, 29452), (351,))

In [4]:
spa_adata.obsm['spatial'] = np.hstack([spa_adata.obs.x_global_affine.values.reshape(-1,1), spa_adata.obs.y_global_affine.values.reshape(-1,1)])
np.save('../../output/seqfish_locations.npy', spa_adata.obsm['spatial'])
sq.gr.spatial_neighbors(spa_adata)


In [5]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)
kf.get_n_splits(raw_shared_gene)

df_transImpSpa = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_transImpCls = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_transImpClsSpa = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_transImp = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_stplus_res = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_spaGE_res = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)
df_tangram_res = pd.DataFrame(np.zeros((spa_adata.n_obs, len(raw_shared_gene))), columns=raw_shared_gene)

for idx, (train_ind, test_ind) in enumerate(kf.split(raw_shared_gene)):    
    print(f"\n===== Fold {idx+1} =====\nNumber of train genes: {len(train_ind)}, Number of test genes: {len(test_ind)}")
    train_gene = raw_shared_gene[train_ind]
    test_gene  = raw_shared_gene[test_ind]
    
    test_spatial_df = raw_spatial_df[test_gene]
    spatial_df = raw_spatial_df[train_gene]
    scrna_df   = raw_scrna_df

    df_transImpSpa[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            signature_mode='cell',
            mapping_mode='lowrank',
            mapping_lowdim=128,
            n_epochs=2000,
            spa_adj=spa_adata.obsp['spatial_connectivities'].tocoo(),
            seed=seed,
            device=device)

    corr_transImp_res = calc_corr(raw_spatial_df, df_transImpSpa, test_gene)
    print(f'fold {idx}, median cosine similarity: {np.median(corr_transImp_res)} (TransImpSpa)')

    df_transImpCls[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            ct_list=ct_list,
            classes=classes,
            signature_mode='cluster',
            mapping_mode='full',
            seed=seed,
            device=device)

    corr_transImp_res = calc_corr(raw_spatial_df, df_transImpCls, test_gene)
    print(f'fold {idx}, median cosine similarity: {np.median(corr_transImp_res)} (TransImpCls)')

    df_transImp[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            signature_mode='cell',
            mapping_mode='lowrank',
            mapping_lowdim=128,
            seed=seed,
            device=device)

    corr_transImp_res = calc_corr(raw_spatial_df, df_transImp, test_gene)
    print(f'fold {idx}, median cosine similarity: {np.median(corr_transImp_res)} (TransImp)')

    df_transImpClsSpa[test_gene] = expTransImp(
            df_ref=raw_scrna_df,
            df_tgt=raw_spatial_df,
            train_gene=train_gene,
            test_gene=test_gene,
            ct_list=ct_list,
            classes=classes,
            spa_adj=spa_adata.obsp['spatial_connectivities'].tocoo(),
            signature_mode='cluster',
            mapping_mode='full',
            seed=seed,
            device=device)

    corr_transImp_res = calc_corr(raw_spatial_df, df_transImpClsSpa, test_gene)
    print(f'fold {idx}, median cosine similarity: {np.median(corr_transImp_res)} (TransImpClsSpa)')

    df_stplus_res[test_gene] = stPlus.stPlus(spatial_df, scrna_df, test_gene, "tmp_ug", verbose=False, random_seed=seed, device=device)
    corr_res_stplus = calc_corr(raw_spatial_df, df_stplus_res, test_gene)
    print(f'fold {idx}, median cosine similarity: {np.median(corr_res_stplus)} (stPlus)')

    df_spaGE_res[test_gene]  = SpaGE_impute(scrna_df.sparse.to_dense(), spatial_df, train_gene, test_gene)
    corr_res_spaGE = calc_corr(raw_spatial_df, df_spaGE_res, test_gene)
    print(f'fold {idx}, median cosine similarity: {np.median(corr_res_spaGE)} (spaGE)')

    df_tangram_res[test_gene] = Tangram_impute(scrna_adata, spa_adata, train_gene, test_gene, device, cls_key)
    corr_res_tangram = calc_corr(raw_spatial_df, df_tangram_res, test_gene)
    print(f'fold {idx}, median cosine similarity: {np.median(corr_res_tangram)} (Tangram)')

corr_transImpSpa_res = calc_corr(raw_spatial_df, df_transImpSpa, raw_shared_gene)
corr_transImp_res = calc_corr(raw_spatial_df, df_transImp, raw_shared_gene)
corr_transImpCls_res = calc_corr(raw_spatial_df, df_transImpCls, raw_shared_gene)
corr_transImpClsSpa_res = calc_corr(raw_spatial_df, df_transImpClsSpa, raw_shared_gene)
corr_res_stplus = calc_corr(raw_spatial_df, df_stplus_res, raw_shared_gene)
corr_res_spaGE = calc_corr(raw_spatial_df, df_spaGE_res, raw_shared_gene)
corr_res_tangram = calc_corr(raw_spatial_df, df_tangram_res, raw_shared_gene)   

print(np.median(corr_transImpSpa_res), "(TransImpSpa)", 
      np.median(corr_transImp_res), "(TransImp)", 
      np.median(corr_transImpCls_res), "(TransImpCls)", 
      np.median(corr_transImpClsSpa_res), "(TransImpClsSpa)", 
      np.median(corr_res_stplus), "(stPlus)", 
      np.median(corr_res_spaGE), "(spaGE)",
      np.median(corr_res_tangram), "(Tangram)"
      )


===== Fold 1 =====
Number of train genes: 280, Number of test genes: 71


[TransImp] Epoch: 2000/2000, loss: 0.756026, (IMP) 0.753120, (SPA) 1.0 x 0.002906: 100%|██████████| 2000/2000 [00:33<00:00, 59.71it/s]


fold 0, median cosine similarity: 0.44133231043815613 (TransImpSpa)


[TransImp] Epoch: 1000/1000, loss: 0.907009, (IMP) 0.907009, (SPA) 1.0 x 0.000000: 100%|██████████| 1000/1000 [00:07<00:00, 126.82it/s]


fold 0, median cosine similarity: 0.4470369219779968 (TransImpCls)


...

fold 4, median cosine similarity: 0.4983830899000168 (TransImpClsSpa)
fold 4, median cosine similarity: 0.5207089283803024 (stPlus)
fold 4, median cosine similarity: 0.5026244338884789 (spaGE)


INFO:root:281 training genes are saved in `uns``training_genes` of both single cell and spatial Anndatas.
INFO:root:281 overlapped genes are saved in `uns``overlap_genes` of both single cell and spatial Anndatas.
INFO:root:uniform based density prior is calculated and saved in `obs``uniform_density` of the spatial Anndata.
INFO:root:rna count based density prior is calculated and saved in `obs``rna_count_based_density` of the spatial Anndata.
INFO:root:Allocate tensors for mapping.
INFO:root:Begin training with 281 genes and rna_count_based density_prior in clusters mode...
INFO:root:Printing scores every 100 epochs.


Score: 0.377, KL reg: 0.068
Score: 0.559, KL reg: 0.001
Score: 0.560, KL reg: 0.001
Score: 0.560, KL reg: 0.001
Score: 0.561, KL reg: 0.001
Score: 0.561, KL reg: 0.001
Score: 0.561, KL reg: 0.001
Score: 0.561, KL reg: 0.001
Score: 0.561, KL reg: 0.001
Score: 0.561, KL reg: 0.001


INFO:root:Saving results..


fold 4, median cosine similarity: 0.4984356164932251 (Tangram)
0.48657330870628357 (TransImpSpa) 0.5000782608985901 (TransImp) 0.48267295956611633 (TransImpCls) 0.47081634402275085 (TransImpClsSpa) 0.48195333174207633 (stPlus) 0.4539831970654655 (spaGE) 0.4765040874481201 (Tangram)


In [6]:
df_transImp.to_csv('../../output/seqFISH_SingleCell_transImpute.csv')
df_transImpSpa.to_csv('../../output/seqFISH_SingleCell_transImpSpa.csv')
df_transImpCls.to_csv('../../output/seqFISH_SingleCell_transImpCls.csv')
df_transImpClsSpa.to_csv('../../output/seqFISH_SingleCell_transImpClsSpa.csv')
df_spaGE_res.to_csv('../../output/seqFISH_SingleCell_spaGE.csv')
df_stplus_res.to_csv('../../output/seqFISH_SingleCell_stPlus.csv')
df_tangram_res.to_csv('../../output/seqFISH_SingleCell_Tangram.csv')


In [7]:
dict_df = {"TransImp":df_transImp, 
           "TransImpSpa":df_transImpSpa, 
           "TransImpCls":df_transImpCls,
           "TransImpClsSpa":df_transImpClsSpa,
           "spaGE": df_spaGE_res, "stPlus": df_stplus_res,
            "Tangram":df_tangram_res
            }
spa_adata.X = spa_adata.X.toarray()
sq.gr.spatial_autocorr(
    spa_adata,
    n_jobs=10,
)

dict_adata = {name: compute_autocorr(spa_adata.copy(), df) for name, df in dict_df.items()}


In [8]:
from sklearn.metrics import mean_squared_error
moranIs = {name:mean_squared_error(spa_adata.uns['moranI'].loc[raw_shared_gene].I, imp_adata.uns['moranI'].loc[raw_shared_gene].I) for name, imp_adata in dict_adata.items()}

print("Mean Squared Error\nMoran's I:\n")
print("\n".join([f"\tTrue vs {method}: {score:.6f}" for method, score in moranIs.items()]))



Mean Squared Error
Moran's I:

	True vs TransImp: 0.057495
	True vs TransImpSpa: 0.011765
	True vs TransImpCls: 0.050609
	True vs TransImpClsSpa: 0.007408
	True vs spaGE: 0.034764
	True vs stPlus: 0.051806
	True vs Tangram: 0.038962
